A Kalman Filter is just a Bayesian Filter that uses Gaussian distributions. Compared to the chapter on discrete Guassian Filters, in this chapter we replace histograms with Gaussians:

$$\begin{array}{l|l|c}
\text{discrete Bayes} & \text{Gaussian} & \text{Step}\\
\hline
\bar {\mathbf x} = \mathbf x \ast f(\mathbf x) & 
\bar {x}_\mathcal{N} =  x_\mathcal{N} \, \oplus \, f_{x_\mathcal{N}}(\bullet) &
\text{Predict} \\
\mathbf x = \|\mathcal L \bar{\mathbf x}\| & x_\mathcal{N} = L \, \times \, \bar{x}_\mathcal{N} & \text{Update} 
\end{array}$$

where $\oplus$ takes two Guassians $\mathcal{N}_1$ and $\mathcal{N}_2$ and produces the Gaussian $\mathcal{N}$ that represents the distibution of $X = X_1 + X_2$ where $X_1 \sim \mathcal{N}_1$ and $X_2 \sim \mathcal{N}_2$ (recall that this is effectively a convolution), and $\times$ represents product of Gaussians.

In [1]:
from collections import namedtuple

gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: f'𝒩(μ={s[0]:.3f}, 𝜎²={s[1]:.3f})'

So our predict and update functions looks like

In [8]:
def predict(pos: gaussian, movement: gaussian):
    return gaussian(pos.mean + movement.mean, pos.var + movement.var)

def gaussian_multiply(g1, g2):
    mean = (g1.var * g2.mean + g2.var * g1.mean) / (g1.var + g2.var)
    variance = (g1.var * g2.var) / (g1.var + g2.var)
    return gaussian(mean, variance)

def update(prior, likelihood):
    posterior = gaussian_multiply(likelihood, prior)
    return posterior


Let's try to use this on the dog

In [32]:
from kf_book.kf_internal import DogSimulation

process_var = 1.  # variance in the dog's movement
velocity = 1  # constant velocity for process model
dt = 1.  # time step to discretize process model
process_model = gaussian(velocity*dt, process_var)  # a constant velocity process model
sensor_var = 2.  # variance in the sensor measurement

# An initial distrubution for the dog's position
x = gaussian(0, 20.**2)  # So basically we're 99.7% sure the dog is within 60 meters of the origin.

dog = DogSimulation(
    x0=x.mean, 
    velocity=process_model.mean, 
    measurement_var=sensor_var, 
    process_var=process_model.var)

# A list of measurements to then be fed into a Kalman filter
ts_z = [dog.move_and_sense() for _ in range(20)]

# Apply Kalman Filter
ts_x_prior = []
ts_x = []
for z in ts_z:
    x_prior = predict(x, process_model)
    ts_x_prior.append(x_prior)
    x = update(x_prior, gaussian(z, sensor_var))
    ts_x.append(x)

import math
table = [["Measurement", "Prior", "Stdev", "Posterior", "Stdev"]] \
         + [[f"{z:.3f}", f"{x_prior.mean:.3f}", f"{math.sqrt(x_prior.var):.3f}", f"{x.mean:.3f}", f"{math.sqrt(x.var):.3f}"]
         for z, x_prior, x in zip(ts_z, ts_x_prior, ts_x)]
from tabulate import tabulate
print(tabulate(table))

-----------  ------  ------  ---------  -----
Measurement  Prior   Stdev   Posterior  Stdev
-0.474       1.000   20.025  -0.467     1.411
-1.688       0.533   1.729   -0.798     1.095
1.801        0.202   1.483   1.040      1.023
3.124        2.040   1.431   2.588      1.006
4.111        3.588   1.418   3.851      1.001
2.483        4.851   1.415   3.666      1.000
3.947        4.666   1.414   4.306      1.000
4.779        5.306   1.414   5.043      1.000
6.059        6.043   1.414   6.051      1.000
6.947        7.051   1.414   6.999      1.000
8.807        7.999   1.414   8.403      1.000
6.439        9.403   1.414   7.921      1.000
11.289       8.921   1.414   10.105     1.000
10.540       11.105  1.414   10.822     1.000
11.708       11.822  1.414   11.765     1.000
12.485       12.765  1.414   12.625     1.000
14.786       13.625  1.414   14.206     1.000
14.437       15.206  1.414   14.821     1.000
14.240       15.821  1.414   15.031     1.000
16.287       16.031  1.414   16.15

Now let's reminder ourselves of the expressions for the mean of product of gaussians:

$$
\begin{aligned}
\mu &=\frac{\sigma_1^2\mu_2 + \sigma_2^2\mu_1}{\sigma_1^2+\sigma_2^2}\\
&=\frac{\sigma_1^2}{\sigma_1^2+\sigma_2^2} \mu_2 + \frac{\sigma_2^2}{\sigma_1^2+\sigma_2^2} \mu_1 \\
&=\frac{\bar\sigma^2}{\bar\sigma^2+\sigma_z^2} \mu_z + \frac{\sigma_z^2}{\bar\sigma^2+\sigma_z^2} \bar\mu
\end{aligned}
$$

where in the last line I've specialized for the Kalman Filter settings with $\bar\cdot$ for the prior and $\cdot_z$ for the measurement. Notice that the coefficents in the second line sum to 1. So when we are updating to get a new estimate we are taking a weighted sum of the sensor mean and the mean of the prior. An easy way to think about it, is that if the variance of the measurement is $r$ times greater than the variance of the prior, we weight the prior $r$ times more than the measurement.

We write the "Kalman gain" as

$$
K = \frac{\bar\sigma^2}{\bar\sigma^2+\sigma_z^2}
$$

we can also express the updated variance in terms of the Kalman gain and measurement variance or prior variance. First recall that for the product of Gaussians:

$$
\begin{aligned}
\sigma^2 &=\frac{\sigma_1^2\sigma_2^2}{\sigma_1^2+\sigma_2^2} 
\end{aligned}
$$

So casting this into the Kalman Filter setting:

$$
\begin{aligned}
\sigma^2 &= \frac{\bar\sigma^2 \sigma_z^2 } {\bar\sigma^2 + \sigma_z^2} \\
&= K\sigma_z^2 \\
&= (1-K)\bar\sigma^2 
\end{aligned}
$$

Yet another useful way of talking about things is that the Kalman gain is a scale factor that chooses a value along the residual (the residual being the line connecting the prior and the measurement).

These equations give rise to the more conventional way of writing a Kalman filter (pay attention to the choice of variable names as these are the most widely used in the literature)

In [33]:
def predict(posterior, movement):
    x, P = posterior # mean and variance of posterior
    dx, Q = movement # mean and variance of movement
    x = x + dx
    P = P + Q
    return gaussian(x, P)


def update(prior, measurement):
    x, P = prior        # mean and variance of prior
    z, R = measurement  # mean and variance of measurement
    
    y = z - x        # residual
    K = P / (P + R)  # Kalman gain (decide how far to interpolate along the residual, small P means we stay close to the prior)

    x = x + K*y      # posterior (interpolation of the residual)
    P = (1 - K) * P  # posterior variance
    return gaussian(x, P)

    

Exercise: Interactive Plots

In [43]:
from numpy.random import seed
from ipywidgets import interact
from kf_book.book_plots import FloatSlider
import matplotlib.pyplot as plt


def plot_kalman_filter(start_pos, 
                       sensor_var, 
                       velocity, 
                       process_var):
    seed(0)
    dt = 1.  # time step to discretize process model
    process_model = gaussian(velocity*dt, process_var)  # a constant velocity process model

    # An initial distrubution for the dog's position
    x = gaussian(start_pos, 20.**2)  # So basically we're 99.7% sure the dog is within 60 meters of the origin.

    dog = DogSimulation(
        x0=x.mean, 
        velocity=process_model.mean, 
        measurement_var=sensor_var, 
        process_var=process_model.var)

    # A list of measurements to then be fed into a Kalman filter
    ts_z = [dog.move_and_sense() for _ in range(20)]

    # Apply Kalman Filter
    ts_x_prior = []
    ts_x = []
    for z in ts_z:
        x_prior = predict(x, process_model)
        ts_x_prior.append(x_prior)
        x = update(x_prior, gaussian(z, sensor_var))
        ts_x.append(x)

    plt.figure()
    plt.plot(ts_z, c='k', marker='o', linestyle='', label='measurement')
    plt.plot([x.mean for x in ts_x_prior], marker='^', linestyle='', c='red', alpha=0.7, label='update')
    plt.plot([x.mean for x in ts_x], c='#004080', alpha=0.7, label='filter')
    plt.legend(loc=4)
    plt.grid()
    plt.show()
    

interact(plot_kalman_filter,
         start_pos=(-10, 10), 
         sensor_var=FloatSlider(value=5, min=0, max=100), 
         velocity=FloatSlider(value=1, min=-2., max=2.), 
         process_var=FloatSlider(value=5, min=0, max=100.));

interactive(children=(IntSlider(value=0, description='start_pos', max=10, min=-10), FloatSlider(value=5.0, con…